In [1]:
import pandas as pd
import nltk
import os

In [2]:
os.listdir()

['.gitignore',
 '.ipynb_checkpoints',
 '01. DMZ 피드백 재시도.ipynb',
 '01. DMZ 피드백 재시도_2.ipynb',
 'UseData',
 '네트워크_dmz_feedback.ipynb']

In [3]:
url = 'UseData/'
os.listdir(url)

['(학술자료_해외) animal.xlsx',
 '(학술자료_해외) Biodiversity.xlsx',
 '(학술자료_해외) plant.xlsx',
 '(학술자료_해외) preservation or conservation.xlsx',
 '(학술자료_해외) south korea.xlsx']

In [4]:
files = pd.DataFrame({
    'FileName' : os.listdir(url),
    'Lang' : ['en', 'en', 'en', 'en', 'en']
})
files

,FileName,Lang
0,(학술자료_해외) animal.xlsx,en
1,(학술자료_해외) Biodiversity.xlsx,en
2,(학술자료_해외) plant.xlsx,en
3,(학술자료_해외) preservation or conservation.xlsx,en
4,(학술자료_해외) south korea.xlsx,en


In [ ]:
# 함수 정의
# 파일을 읽고, 어간 별 분리를 한 후 갯수를 count, gephi에 넣을 수 있게 가공.
def Processor(url, filename, lang):
    df = pd.read_excel(url + filename, sheet_name = 'Sheet1', usecols=['Contents'])
    
    resultList = []
    countDict = {}
    
    # 숫자 제거, 길이가 3 이하인 단어만 추출 및 lower 전처리